<a href="https://colab.research.google.com/github/valmirf/mineracao_textual/blob/main/CNN/Exercicio_Analise_Sentimentos_Imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Análise de Sentimentos
A análise de sentimento visa determinar a opinião ou sentimento das pessoas eem relçao a um produto, pessoa, evento, ou qualquer outro objeto de interesse. Por exemplo, um palestrante ou escritor em relação a um documento, interação ou evento. O sentimento é principalmente categorizado em categorias positivas, negativas e neutras. Por meio da análise de sentimento, podemos prever, por exemplo, a opinião e a atitude de um cliente sobre um produto com base em uma resenha que ele escreveu. Essa técnica é amplamente aplicada a coisas como revisões, pesquisas, documentos e muito mais.

##Base de Dados IMDB
O conjunto de dados de classificação de sentimento do IMDB (https://www.imdb.com/) consiste em 50000 resenhas de filmes de usuários do IMDB que são rotuladas como positivas (1) ou negativas (0). As 50000 resenhas são divididas em 25000 para treinamento e 25000 para teste. O conjunto de dados foi criado por pesquisadores da Universidade de Stanford e publicado em um artigo de 2011, onde alcançou 88.89% de precisão. 

In [ ]:
from keras.datasets import imdb

# Load the data, keeping only 10,000 of the most frequently occuring words
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

In [ ]:
import numpy as np

# Since we restricted ourselves to the top 10000 frequent words, no word index should exceed 10000
# we'll verify this below
# Here is a list of maximum indexes in every review --- we search the maximum index in this list of max indexes
print(type([max(sequence) for sequence in train_data]))

# Find the maximum of all max indexes
max([max(sequence) for sequence in train_data])

data = np.concatenate((train_data, test_data), axis=0)
labels = np.concatenate((train_labels, test_labels), axis=0)


print("Categories:", np.unique(labels))
print("Number of unique words:", len(np.unique(np.hstack(data))))

length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

Você pode ver acima que o conjunto de dados é classificado em duas categorias, 0 ou 1, que representa o sentimento da avaliação do filme, negativo e positivo respectivamente. Todo o conjunto de dados contém 9.998 palavras únicas e o comprimento médio da revisão é de 234 palavras, com um desvio padrão de 173 palavras. 

##Decodificação da Avaliação
Abaixo, vamos decodificar o rótulo no formato *One-Hot Code*.  


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(labels.reshape(-1, 1))
print(y[0])


##Preparação dos Dados

Nos códigos abaixo iremos realizar a separação dos dados em treinamento e teste utilizando a função `train_test_split` do sklearn. 

In [ ]:
sentences_train, sentences_test, y_train, y_test  = train_test_split(data, y, test_size=0.3, random_state=42, stratify=y)
print(sentences_train[:2], y_train[:2])

## Sequence Padding

Um problema que temos é que cada sequência de texto tem na maioria dos casos diferentes comprimentos de palavras. Para corrigir isso, vamos usar `pad_sequence()` que simplesmente preenche a sequência de palavras com zeros. O número máximo 250 foi escolhido por uma aproximação da média de palavras na avaliação que é 234. 

In [ ]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 250
X_train = pad_sequences(sentences_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(sentences_test, padding='post', maxlen=maxlen)
print(X_train[100])


##Construindo a Rede Neural

Agora podemos construir uma rede neural simples. Começaremos definindo o tipo de modelo que queremos construir. Existem dois tipos de modelos disponíveis no Keras: o modelo sequencial e a classe do modelo usado na API funcional.

Começamos adicionando a primeira camada padrão ao lidar com Texto e CNNs: a camada de Embeddings. Nesse exercício vamos usar um Embedding de tamanho 300. Depois, camadas de Convolução e MaxPooling são adicionadas e por fim, camadas de classificação Densas, sendo que a última camada utiliza a função de classificaçao Softmax.
Por último, vemos um resumo do modelo que acabamos de construir. OBS: Lembre-se de mudar seu ambiente de execução para GPU.

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import models
from keras import optimizers
from keras import losses

embedding_dim = 300
num_classes = y_train.shape[1]


model = Sequential()
vocab_size = X_train.shape[0] + 1 

model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Conv1D(32, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

#mostra o modelo construído
model.summary()

##Compilação do Modelo

Nessa parte do código, definimos os algoritmos de otimização, a função de perda e a métrica que será utilizada para avaliação do modelo. 

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss = losses.binary_crossentropy,
              metrics = ['accuracy'])


##Configuração da Avaliação
Reservaremos uma parte de nossos dados de treinamento para validação da precisão do modelo durante o treinamento. Um conjunto de validação nos permite monitorar o progresso de nosso modelo em dados não vistos anteriormente à medida que ele passa por épocas durante o treinamento.
As etapas de validação nos ajudam a ajustar os parâmetros de treinamento da função `model.fit` para evitar overfitting e underfitting de dados.


In [ ]:
# Input for Validation
X_val = X_train[:5000]
partial_X_train = X_train[10000:]

# Labels for validation
y_val = y_train[:5000]
partial_y_train = y_train[10000:]

##Treinamento do Modelo

In [ ]:
history = model.fit(partial_X_train, 
                    partial_y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_val, y_val),
                    batch_size=512)


### Resultados

In [ ]:
from sklearn import metrics

loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

preds = model.predict_classes(X_test)
true  = np.argmax(y_test, axis=1)

print(metrics.classification_report(true, preds))

##Visualizando os Resultados

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

history_dict = history.history
history_dict.keys()

# Plotting losses
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values) + 1)

plt.plot(epochs, loss_values, 'bo', label="Training Loss")
plt.plot(epochs, val_loss_values, 'b', label="Validation Loss")

plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss Value')
plt.legend()

plt.show()



## Adicionando embeddings pré-treinados
É possível usarmos embeddings pré-treinados. A escolha é sempre relativa ao seu problema. Por exemplo, se você precisa resolver um problema de classificação de texto de cunho geral, pode pegar um Embeddings pré-treinado do Google, com milhões de textos. Porém, se quiser resolver um problema de classificação de sentimentos de review de livros, pode ser útil utilizar um embeddings mais próximo do seu problema, como por exemplo, um embeddings pré-treinado com informações e review de livros da Amazon. Nessa atividade, vamos utilizar o Embedding de notícias do Google.
### Word2Vec

In [ ]:
!wget https://gist.githubusercontent.com/bastings/4d1c346c68969b95f2c34cfbc00ba0a0/raw/76b4fefc9ef635a79d0d8002522543bc53ca2683/googlenews.word2vec.300d.txt

### GLove

In [ ]:
!wget https://gist.githubusercontent.com/bastings/b094de2813da58056a05e8e7950d4ad1/raw/3fbd3976199c2b88de2ae62afc0ecc6f15e6f7ce/glove.840B.300d.sst.txt

In [ ]:
import numpy as np

def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    
    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix
    


#Exercícios

1. Realize a análise de sentimentos com os Embeddings pré-treinados Word2Vec e Glove, utilizando a rede neural criada em cima. Na tabela abaixo, troque o X pelos resultados da acurácia, do precision, do recall e F1.

\begin{array}{|c|c|c|c|c|}\hline 
  Embedding & Precisão & Recall & F1 & Acurácia\\  \hline 
Word2Vec & X & X & X & X  \\ \hline
Glove  & X & X & X & X  \\ \hline 
\end{array}

In [ ]:
from keras.models import Sequential
from keras import layers
from keras import models
from keras import optimizers
from keras import losses
from sklearn import metrics

#recupera palavras a partir dos indices da base imdb
word_index = imdb.get_word_index()
index_word = {k:v for k,v in word_index.items()}

#Cria a matriz de Embeddings
embedding_matrix = create_embedding_matrix('googlenews.word2vec.300d.txt',
                      index_word, embedding_dim)

embedding_dim = 300
num_classes = y_train.shape[1]

#Cria Rede Neural
modelWord2Vec = Sequential()
vocab_size = X_train.shape[0] + 1 

modelWord2Vec.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
modelWord2Vec.add(layers.Conv1D(32, 5, activation='relu'))
modelWord2Vec.add(layers.GlobalMaxPooling1D())
modelWord2Vec.add(layers.Dense(10, activation='relu'))
modelWord2Vec.add(layers.Dense(num_classes, activation='softmax'))

#mostra o modelo construído
#modelWord2Vec.summary()

#Compila o Modelo 
modelWord2Vec.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss = losses.binary_crossentropy,
              metrics = ['accuracy'])

#Treina o Modelo
history = modelWord2Vec.fit(partial_X_train, 
                    partial_y_train,
                    epochs=10,
                    verbose=False,
                    validation_data=(X_val, y_val),
                    batch_size=512)

#Avalia o Modelo
loss, accuracy = modelWord2Vec.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = modelWord2Vec.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

preds = modelWord2Vec.predict_classes(X_test)
true  = np.argmax(y_test, axis=1)

print(metrics.classification_report(true, preds))

2. Agora, crie uma nova rede neural e altere seus parâmetros para melhorar os resultados obtidos na questão anterior para os Embeddings pré-treinados Word2Vec e Glove. Mantenha o número de épocas em 10. 

  a) Escreva qual foi sua melhor alteração?  
  b) Na tabela abaixo, troque o X pelos melhores resultados da acurácia, do precision, do recall e F1.

\begin{array}{|c|c|c|c|c|}\hline 
  Embedding & Precisão & Recall & F1 & Acurácia\\  \hline 
Word2Vec & X & X & X & X  \\ \hline
Glove  & X & X & X & X  \\ \hline 
\end{array}